`(1) Env 환경변수`

In [17]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
print(TAVILY_API_KEY[:4])

sk
z6
tvly


`(2) 기본 라이브러리`

In [18]:
import warnings
warnings.filterwarnings("ignore")

from langchain_community.vectorstores import FAISS
from langchain_core.messages import SystemMessage
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
#from langchain_core.tools import tool
from langchain.agents import tool
from langchain_community.tools import TavilySearchResults
from langchain_core.documents import Document

from langchain_openai import ChatOpenAI
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage

# LangGraph MessagesState라는 미리 만들어진 상태를 사용
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import create_react_agent

from textwrap import dedent
from typing import List, Literal, Tuple, TypedDict
from pydantic import BaseModel, Field

import gradio as gr

from pprint import pprint

import uuid

#from IPython.display import Image, display

###  2-1. Tool 정의

- 메뉴 검색을 위한 벡터저장소를 초기화 (기존 저장소를 로드)

In [19]:

embeddings_model = UpstageEmbeddings(model="solar-embedding-1-large")

# menu db 벡터 저장소 로드
menu_db = FAISS.load_local(
    "../db/menu_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

# wine db 벡터 저장소 로드
wine_db = FAISS.load_local(
    "../db/wine_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def search_menu(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=6)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

@tool
def search_wine(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant wine information from the encrypted database.
    Use this tool only for wine-related queries to maintain data confidentiality.
    """
    docs = wine_db.similarity_search(query, k=6)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

# 웹 검색 
@tool
def search_web(query: str) -> List[str]:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = []
    for doc in docs:
        formatted_docs.append(
            Document(
                page_content= f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>',
                metadata={"source": "web search", "url": doc["url"]}
                )
        )

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return [Document(page_content="관련 정보를 찾을 수 없습니다.")]


# 도구 목록을 정의 
tool_list = [search_menu, search_wine, search_web]
print(type(tool_list[0]))

<class 'langchain_core.tools.structured.StructuredTool'>


### 2-2. LLM 모델
* bind_tools() 함수로 model 과 tool 연결

In [20]:
#from langchain_openai import ChatOpenAI

# 기본 LLM
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, streaming=True)

from langchain_upstage import ChatUpstage

llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
print(llm.model_name)

# LLM에 도구 바인딩하여 추가 
llm_with_tools = llm.bind_tools(tool_list)
print(type(llm_with_tools))

solar-pro
<class 'langchain_core.runnables.base.RunnableBinding'>


In [21]:
# 메뉴 검색에 관련된 질문을 하는 경우 -> 메뉴 검색 도구를 호출  
query = "대표 메뉴는 무엇인가요?"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='[The `search_menu` function is ESSENTIAL because the question directly asks for the "대표 메뉴" (representative menu), which requires querying the restaurant\'s authorized menu database. No other function or general knowledge can provide this specific information.]', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-aea32e63ead0499f948421bac81d5133', 'function': {'arguments': '{"query": "\\ub300\\ud45c \\uba54\\ub274"}', 'name': 'search_menu'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 663, 'total_tokens': 730, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '690e067b-5c8b-45e5-b58d-3c44563626bb', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, 

In [22]:
# 도구들의 목적과 관련 없는 질문을 하는 경우 -> 도구 호출 없이 그대로 답변을 생성 
query = "안녕하세요?"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='안녕하세요! 어떻게 도와드릴 수 있을까요? 특정 질문이나 요청사항이 있으시면 알려주세요. 메뉴, 와인 정보 또는 일반 웹 검색 관련 질의에 대해 도움을 드릴 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 660, 'total_tokens': 694, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '71f3ba3b-59c8-49bf-b7f4-a28f4bb9c922', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--978f46b8-ae04-4751-af07-584fd41a6edd-0', usage_metadata={'input_tokens': 660, 'output_tokens': 34, 'total_tokens': 694, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})
----------------------------------------------------------------------------------------------------
('안녕하세요! 어떻게 도와드릴 수 있을까

In [23]:
# 웹 검색 목적과 관련된 질문을 하는 경우 -> 웹 검색 도구 호출 
query = "2024년 상반기 엔비디아 시가총액은 어떻게 변동 되었나요?"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='[엔비디아의 2024년 상반기 시가총액 변동에 대한 최신 데이터는 내부 데이터베이스에 없으므로, 신뢰할 수 있는 외부 소스(금융 뉴스, 주식 시장 플랫폼 등)에서 정보를 검색해야 합니다. `search_web` 함수는 이러한 실시간/최신 정보를 제공하는 데 필수적입니다.]', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-f092e3df077e48bcbb97ab668ed5dd54', 'function': {'arguments': '{"query": "2024\\ub144 \\uc0c1\\ubc18\\uae30 \\uc5d4\\ube44\\ub514\\uc544 \\uc2dc\\uac00\\ucd1d\\uc561 \\ubcc0\\ub3d9"}', 'name': 'search_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 675, 'total_tokens': 757, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '2ea46173-4742-4e29-888a-c7e14a61de3d', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b86833ba-7c32-4233-8505-

## 3. Adaptive RAG


### 3-1. 그래프 구현

`(1) 상태 정의`

In [24]:
# 상태 Schema 정의 
class AdaptiveRagState(TypedDict):
    question: str
    documents: List[Document]
    generation: str

`(2) 질문 분석 -> 라우팅`
- 사용자의 질문을 분석하여 적절한 검색 방법을 선택 
- 레스토랑 메뉴 검색 or 레스토랑 와인 검색  or 일반 웹 검색 or 단순 답변

In [25]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from textwrap import dedent

# 1. 출력 구조 정의 (Pydantic Model)
# LLM의 출력이 따르기를 원하는 JSON 스키마를 Pydantic 모델로 정의합니다.
class ToolSelector(BaseModel):
    """사용자 질문을 가장 적절한 도구로 라우팅하는 역할의 데이터 모델."""
    
    # tool 필드를 정의합니다. Literal을 사용하여 이 필드의 값은
    # 'search_menu', 'search_web', 'search_wine' 셋 중 하나여야 함을 강제합니다.
    # 이는 LLM이 잘못된 도구 이름을 생성하는 것을 방지합니다.
    tool: Literal["search_menu", "search_web", "search_wine"] = Field(
        description="사용자의 질문을 기반으로 search_menu, search_wine, search_web 중 하나의 도구를 선택하세요.",
    )

# 2. 구조화된 출력을 위한 LLM 설정 (with_structured_output 핵심 적용)
# 기존 LLM에 with_structured_output을 적용하여 새로운 structured_llm을 만듭니다.
# 이 함수는 LLM이 ToolSelector 클래스가 정의한 구조(JSON 스키마)를 따르는 객체를 출력하도록 강제합니다.
structured_llm = llm.with_structured_output(ToolSelector)
print(type(structured_llm)) # 출력 타입: LangChain Runnable

# 3. 라우팅을 위한 프롬프트 템플릿
# dedent를 사용하여 여러 줄의 시스템 프롬프트를 깔끔하게 정의합니다.
system = dedent("""당신은 사용자 질문을 적절한 도구로 라우팅하는 데 특화된 AI 어시스턴트입니다.
다음 지침을 사용하십시오:
- 레스토랑의 메뉴에 대한 질문에는 search_menu 도구를 사용하십시오.
- 와인 추천이나 페어링 정보에는 search_wine 도구를 사용하십시오.
- 기타 다른 정보나 최신 데이터에 대한 질문에는 search_web 도구를 사용하십시오.
항상 사용자 질문을 기반으로 가장 적절한 도구를 선택하십시오.""")

# 시스템 역할과 사용자 질문을 포함하는 ChatPromptTemplate을 생성합니다.
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"), # 사용자의 실제 질문이 여기에 삽입됩니다.
    ]
)

# 4. 질문 라우터 정의 (체인 구성)
# 프롬프트와 구조화된 LLM을 연결하여 라우팅 체인(question_router)을 완성합니다.
# 이 체인을 실행하면, LLM은 프롬프트의 지침에 따라 ToolSelector 객체를 반환합니다.
question_router = route_prompt | structured_llm
print(type(question_router)) # 출력 타입: LangChain Runnable

# 5. 테스트 실행
# 'invoke'를 사용하여 각기 다른 질문에 대한 라우팅 결과를 확인합니다.
print(question_router.invoke({"question": "채식주의자를 위한 메뉴가 있나요?"}))
# 예상 출력: tool='search_menu' (메뉴에 대한 질문)

print(question_router.invoke({"question": "스테이크 메뉴와 어울리는 와인을 추천해주세요."}))
# 예상 출력: tool='search_wine' (와인 추천에 대한 질문)

print(question_router.invoke({"question": "2022년 월드컵 우승 국가는 어디인가요?"}))
# 예상 출력: tool='search_web' (일반적인 최신 정보에 대한 질문)

<class 'langchain_core.runnables.base.RunnableSequence'>
<class 'langchain_core.runnables.base.RunnableSequence'>
tool='search_menu'
tool='search_wine'
tool='search_web'


In [26]:
# 질문 라우팅 노드 
# 상태 Schema 정의  class AdaptiveRagState(TypedDict):
def route_question_adaptive(state: AdaptiveRagState) -> Literal["search_menu", "search_wine", "search_web", "llm_fallback"]:
    question = state["question"]
    try:
        result = question_router.invoke({"question": question})
        datasource = result.tool
        
        if datasource == "search_menu":
            return "search_menu"
        elif datasource == "search_wine":
            return "search_wine"        
        elif datasource == "search_web":
            return "search_web"
        else:
            return "llm_fallback"
    
    except Exception as e:
        print(f"Error in routing: {str(e)}")
        return "llm_fallback"

`(3) 검색 노드`

In [27]:
# 'AdaptiveRagState'는 LangGraph의 상태(State) 딕셔너리 타입을 나타내는 것으로 추정됩니다.
# 이 딕셔너리는 일반적으로 "question", "documents" 등의 키를 포함합니다.
from typing import TypedDict, List
from langchain_core.documents import Document

# AdaptiveRagState 타입을 정의
# 여기서는 예시를 위해 최소한의 구조만 정의합니다.
# class AdaptiveRagState(TypedDict):
#     """LangGraph의 상태를 정의하는 딕셔너리."""
#     question: str
#     documents: List[Document]
#     # 기타 상태 필드...

# search_menu, search_wine, search_web은 실제 검색 엔진(예: Retriever)을 호출하는 Runnable 객체여야 합니다.
# (이 객체들은 코드 외부에서 정의되어 있어야 합니다.)
# search_menu = ... 
# search_wine = ...
# search_web = ...

def search_menu_adaptive(state: AdaptiveRagState):
    """
    레스토랑 메뉴 내 정보를 검색하는 노드입니다.

    Args:
        state (AdaptiveRagState): 현재 LangGraph의 상태 딕셔너리.
            여기에 'question' 키를 통해 사용자 질문이 포함되어 있습니다.

    Returns:
        dict: 업데이트된 상태 딕셔너리. 검색된 'documents' 리스트를 포함합니다.
    """
    # 1. 상태에서 사용자 질문을 추출합니다.
    question = state["question"]
    
    # 2. 메뉴 검색 도구(Retriever)를 호출하여 질문과 관련된 문서를 검색합니다.
    # search_menu는 보통 LangChain의 Retriever 객체입니다.
    docs = search_menu.invoke(question)

    # 3. 검색 결과(문서 리스트)를 확인하고 상태를 반환합니다.
    if len(docs) > 0:
        # 관련 문서가 발견된 경우, 해당 문서를 'documents' 키로 반환합니다.
        return {"documents": docs}
    else:
        # 관련 문서가 없는 경우, 문서가 없음을 알리는 메시지를 담은 Document 객체를 반환합니다.
        return {"documents": [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]}


def search_wine_adaptive(state: AdaptiveRagState):
    """
    레스토랑의 와인 리스트 내 정보를 검색하는 노드입니다.

    Args:
        state (AdaptiveRagState): 현재 LangGraph의 상태 딕셔너리.

    Returns:
        dict: 업데이트된 상태 딕셔너리. 검색된 'documents' 리스트를 포함합니다.
    """
    # 1. 상태에서 사용자 질문을 추출합니다.
    question = state["question"]
    
    # 2. 와인 검색 도구(Retriever)를 호출하여 질문과 관련된 와인 문서를 검색합니다.
    docs = search_wine.invoke(question)

    # 3. 검색 결과(문서 리스트)를 확인하고 상태를 반환합니다.
    if len(docs) > 0:
        # 관련 문서가 발견된 경우, 해당 문서를 'documents' 키로 반환합니다.
        return {"documents": docs}
    else:
        # 관련 문서가 없는 경우, 문서가 없음을 알리는 메시지를 담은 Document 객체를 반환합니다.
        return {"documents": [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]}


def search_web_adaptive(state: AdaptiveRagState):
    """
    메뉴와 와인 정보 이외의 일반적인 정보나 최신 정보를 웹에서 검색하고 결과를 반환하는 노드입니다.

    Args:
        state (AdaptiveRagState): 현재 LangGraph의 상태 딕셔너리.

    Returns:
        dict: 업데이트된 상태 딕셔너리. 검색된 'documents' 리스트를 포함합니다.
    """
    # 1. 상태에서 사용자 질문을 추출합니다.
    question = state["question"]
    
    # 2. 웹 검색 도구(Tool)를 호출하여 질문과 관련된 정보를 검색합니다.
    docs = search_web.invoke(question)
    
    # 3. 검색 결과(문서 리스트)를 확인하고 상태를 반환합니다.
    if len(docs) > 0:
        # 관련 문서가 발견된 경우, 해당 문서를 'documents' 키로 반환합니다.
        return {"documents": docs}
    else:
        # 관련 문서가 없는 경우, 문서가 없음을 알리는 메시지를 담은 Document 객체를 반환합니다.
        return {"documents": [Document(page_content="관련 정보를 찾을 수 없습니다.")]}

`(4) 생성 노드`

In [28]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# (AdaptiveRagState, llm 객체 등은 코드 외부에서 정의되었다고 가정합니다.)

# RAG 프롬프트 정의
# ChatPromptTemplate은 LLM에게 전달될 최종 메시지 형식을 정의합니다.
rag_prompt = ChatPromptTemplate.from_messages([
    # 시스템 프롬프트: LLM에게 답변 생성 규칙과 역할을 부여합니다.
    ("system", """당신은 제공된 문서를 기반으로 질문에 답변하는 Assistant입니다. 다음 지침을 따르십시오:

    1. 제공된 문서의 정보만을 사용하십시오.
    2. 문서에 관련 정보가 부족하면 "제공된 문서에는 이 질문에 답변할 정보가 포함되어 있지 않습니다."라고 말하십시오.
    3. 답변에 문서의 관련 부분을 인용하십시오.
    4. 추측하거나 문서에 없는 정보를 추가하지 마십시오.
    5. 답변을 간결하고 명확하게 유지하십시오.
    6. 관련 없는 정보는 생략하십시오."""
    ),
    # 사용자(Human) 프롬프트: 검색된 문서와 실제 질문을 LLM에게 전달하는 템플릿입니다.
    # {documents}와 {question}이 최종적으로 검색된 내용과 사용자 질문으로 대체됩니다.
    ("human", "다음 문서를 사용하여 다음 질문에 답하십시오:\n\n[문서]\n{documents}\n\n[질문]\n{question}"),
])

# RAG 답변 생성 노드 함수
def generate_adaptive(state: AdaptiveRagState):
    """
    이전 노드에서 검색된 문서를 사용하여 최종 답변을 생성하는 노드입니다.

    Args:
        state (AdaptiveRagState): 현재 LangGraph의 상태 딕셔너리. 
            'question' (질문)과 'documents' (검색된 문서)를 포함합니다.

    Returns:
        dict: 업데이트된 상태 딕셔너리. 생성된 답변('generation')을 포함합니다.
    """
    # 1. 상태(state)에서 필요한 정보(질문, 문서)를 안전하게 추출합니다.
    question = state.get("question", None)
    documents = state.get("documents", [])
    
    # 2. 'documents'가 단일 객체일 경우(LangGraph에서 종종 발생) 리스트로 변환하여 처리 일관성을 확보합니다.
    if not isinstance(documents, list):
        documents = [documents]

    # 3. 검색된 문서 리스트를 LLM 프롬프트에 삽입할 수 있는 단일 문자열로 변환합니다.
    # 각 문서의 본문(page_content)과 메타데이터(metadata)를 구분하여 문자열을 만듭니다.
    documents_text = "\n\n".join\
        ([f"---\n본문: {doc.page_content}\n메타데이터:{str(doc.metadata)}\n---" for doc in documents])

    # 4. RAG 체인 구성 및 실행
    # 프롬프트 템플릿(rag_prompt) -> LLM 호출(llm) -> 문자열 파서(StrOutputParser) 순으로 연결합니다.
    rag_chain = rag_prompt | llm | StrOutputParser()
    
    # 구성된 체인에 필요한 변수(documents_text, question)를 전달하여 LLM을 호출합니다.
    generation = rag_chain.invoke({"documents": documents_text, "question": question})
    
    # 5. 생성된 답변을 'generation' 키로 상태 딕셔너리에 추가하여 반환합니다.
    return {"generation": generation}

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# (AdaptiveRagState, llm 객체 등은 코드 외부에서 정의되었다고 가정합니다.)

# LLM Fallback 프롬프트 정의
# 이 프롬프트는 RAG 문서 없이 LLM의 일반 지식에 의존하여 답변할 때 사용됩니다.
fallback_prompt = ChatPromptTemplate.from_messages([
    # 시스템 프롬프트: 일반 AI 어시스턴트로서의 역할을 정의하고 답변 지침을 부여합니다.
    ("system", """당신은 다양한 주제를 돕는 AI 어시스턴트입니다. 다음 지침을 따르십시오:

    1. 능력이 닿는 한 정확하고 유용한 정보를 제공하십시오.
    2. 확신이 없을 때는 불확실성을 표현하십시오. 추측은 피하십시오.
    3. 답변은 간결하지만 정보를 담고 있도록 유지하십시오.
    4. 사용자에게 필요하면 명확한 설명을 요청할 수 있다고 알리십시오.
    5. 윤리적이고 건설적으로 응답하십시오.
    6. 해당되는 경우 신뢰할 수 있는 일반 출처를 언급하십시오."""),
    # 사용자(Human) 프롬프트: 단순하게 사용자 질문만을 LLM에 전달합니다.
    # 이 프롬프트는 {documents} 변수가 없다는 것이 RAG 프롬프트와의 주요 차이점입니다.
    ("human", "{question}"),
])

# LLM Fallback 답변 생성 노드 함수
def llm_fallback_adaptive(state: AdaptiveRagState):
    """
    검색된 문서(context) 없이 LLM의 일반적인 지식을 사용하여 답변을 생성하는 노드입니다.
    이는 RAG가 실패했을 때의 최종 대안(fallback)을 제공하는 역할을 합니다.

    Args:
        state (AdaptiveRagState): 현재 LangGraph의 상태 딕셔너리. 
            'question' 키를 통해 사용자 질문을 가져옵니다.

    Returns:
        dict: 업데이트된 상태 딕셔너리. LLM이 생성한 답변('generation')을 포함합니다.
    """
    # 1. 상태(state)에서 사용자 질문을 추출합니다.
    question = state.get("question", "")
    
    # 2. LLM 체인 구성
    # Fallback 프롬프트 -> LLM 호출(llm) -> 문자열 파서(StrOutputParser) 순으로 연결합니다.
    llm_chain = fallback_prompt | llm | StrOutputParser()
    
    # 3. 구성된 체인에 질문을 전달하여 LLM을 호출하고 답변을 생성합니다.
    # 이 호출은 RAG 문서 없이 순수하게 LLM의 학습된 지식만을 사용합니다.
    generation = llm_chain.invoke({"question": question})
    
    # 4. 생성된 답변을 'generation' 키로 상태 딕셔너리에 추가하여 반환합니다.
    return {"generation": generation}

`(5) 그래프 연결`

In [30]:
from langgraph.graph import StateGraph, START, END
#from IPython.display import Image, display

# 그래프 구성 시작
# StateGraph를 초기화합니다. 이 객체는 LangGraph 워크플로우의 설계도 역할
# AdaptiveRagState는 이 그래프 전체에서 공유되고 업데이트될 데이터 구조(상태)의 스키마를 정의
builder = StateGraph(AdaptiveRagState)

# 1. 노드 추가 (Add Nodes)
# LangGraph의 각 노드는 워크플로우 내에서 특정 작업을 수행하는 함수를 나타냅니다.
builder.add_node("search_menu", search_menu_adaptive) # 레스토랑 메뉴 검색 노드
builder.add_node("search_wine", search_wine_adaptive) # 와인 리스트 검색 노드
builder.add_node("search_web", search_web_adaptive)   # 일반 웹 검색 노드
builder.add_node("generate", generate_adaptive)       # 검색된 문서를 기반으로 답변을 생성하는 RAG 노드
builder.add_node("llm_fallback", llm_fallback_adaptive) # 검색 실패 시 LLM의 일반 지식으로 답변하는 폴백 노드

# 2. 엣지 추가 (Add Edges)
# 엣지는 노드 간의 흐름을 정의합니다.

# 조건부 엣지 추가 (Conditional Edges)
# 그래프의 시작점(START)에서 다음 노드로의 이동을 결정하는 조건부 라우터를 설정합니다.
builder.add_conditional_edges(
    START,
    # 'route_question_adaptive' 함수를 라우터로 사용합니다. 
    # 이 함수는 사용자 질문을 분석하여 'search_menu', 'search_wine', 'search_web', 'llm_fallback' 중 하나의 노드 이름을 반환
    route_question_adaptive,
    # mapping은 route_question_adaptive의 출력(노드 이름)과 실제 노드를 연결합니다.
    # 이 예시에서는 route_question_adaptive가 노드 이름을 직접 반환한다고 가정하므로, 
    # 추가적인 매핑은 생략되었거나, route_question_adaptive 함수 자체 내에 로직이 포함되어 있습니다.
)

# 일반 엣지 추가 (Fixed Edges)
# 검색 노드가 완료되면, 그 결과(documents)는 답변 생성 노드('generate')로 무조건 이동합니다.
builder.add_edge("search_menu", "generate")
builder.add_edge("search_wine", "generate")
builder.add_edge("search_web", "generate")

# 답변 생성 노드가 완료되면, 그래프 실행을 종료합니다.
builder.add_edge("generate", END)

# LLM Fallback 노드가 완료되어도, 그것이 최종 답변이므로 그래프 실행을 종료합니다.
builder.add_edge("llm_fallback", END)

# 3. 그래프 컴파일 (Compile Graph)
# 정의된 노드와 엣지를 기반으로 실행 가능한 LangChain Runnable 객체로 그래프를 빌드합니다.
# 이 컴파일된 객체('adaptive_rag')는 .invoke() 메서드를 사용하여 실제 워크플로우를 실행합니다.
adaptive_rag = builder.compile()

# 그래프 시각화
#display(Image(adaptive_rag.get_graph().draw_mermaid_png()))

In [31]:
mermaid_code = adaptive_rag.get_graph().draw_mermaid()
print("Mermaid Code:")
print(mermaid_code)

Mermaid Code:
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	search_menu(search_menu)
	search_wine(search_wine)
	search_web(search_web)
	generate(generate)
	llm_fallback(llm_fallback)
	__end__([<p>__end__</p>]):::last
	__start__ -.-> llm_fallback;
	__start__ -.-> search_menu;
	__start__ -.-> search_web;
	__start__ -.-> search_wine;
	search_menu --> generate;
	search_web --> generate;
	search_wine --> generate;
	generate --> __end__;
	llm_fallback --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



* https://mermaid.live/ 에서  mermain_code 로 직접 확인한다.

* [Graph이미지](https://mermaidchart.com/play?utm_source=mermaid_live_editor&utm_medium=share#pako:eNp9kt1ygjAQhV-F2d7gjFgMKhg63tRH6FVLx4mwEaYhMCFMfxzfvStVirXtFbv5djfnLNlDWmUIHHZG1LnzsI4TndjNprHC0Md9uqtXfXZ3W6-eR5xzWZjGHgsbFCbNNyXq1h3EowF7LTS6g_iC4db9DjuyQ41GWHTPQXeqVLmRQqmtSF_cYTL6kos668V2cS9ViS-lvQnHm3grZzgj_oUPzPyDSfR_lNzGP7bkeMTP1uLLVfzNaNA1PCcdOdmOfy7rCqa0kGaN0slQilZZRxZK8RvJpC_lWNFNXo7FLrd8OmEXDd1P78q9qhZpYd-5f1FwXPVp3FZuFzKFMT2rIgNOahocQ4mmFMcc9ol2nARsjiUmwCk8yUkg0Qfqq4V-rKoSuDUtdZqq3eXnpK0zsr0uBL3Zsh9uyCOa-6rVFjibdyOA7-ENeEhWwsBn82C5mAazJcF34NNZNIkWLFqyeRTMQhYdxvDR3elPwjBgIWPTRcB8PwijwyfsSxe-)

In [32]:
# 그래프 실행
inputs = {"question": "스테이크 메뉴의 가격은 얼마인가요?"}
for output in adaptive_rag.stream(inputs):
    for key, value in output.items():
        print(f"Node '{key}':")
        print(f"State '{value.keys()}':")
        print(f"Value '{value}':")
    print("\n---\n")

# 최종 답변
print(value["generation"])

Node 'search_menu':
State 'dict_keys(['documents'])':
Value '{'documents': [Document(id='016ce496-a82b-43f0-bbac-49e2321459ef', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 1, 'menu_name': '시그니처 스테이크'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(id='d6f94fb3-f07f-4524-b9c7-65f2dbc3e102', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 8, 'menu_name': '안심 스테이크 샐러드'}, page_content='8. 안심 스테이크 샐러드\n   • 가격: ₩26,000\n   • 주요 식재료: 소고기 안심, 루꼴라, 체리 토마토, 발사믹 글레이즈\n   • 설명: 부드러운 안심 스테이크를 얇게 슬라이스하여 신선한 루꼴라 위에 올린 메인 요리 샐러드입니다. 체리 토마토와 파마산 치즈 플레이크로 풍미를 더하고, 발사믹 글레이즈로 마무리하여 고기의 풍미를 한층 끌어올렸습니다.'), Document(id='031767b9-b03e-4151-bf25-715429eeb5c7', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 2, 'menu_name': '트러플 리조또'}, page_content='2

In [33]:
# 그래프 실행
inputs = {"question": "푸이 퓌세 2019의 주요 품종은 무엇인가요?"}
for output in adaptive_rag.stream(inputs):
    for key, value in output.items():
        print(f"Node '{key}':")
        print(f"State '{value.keys()}':")
        pprint(f"Value '{value}':")
        #pprint(f"Value '{value.page_content}':")
    print("\n---\n")

# 최종 답변
print(value["generation"])

Node 'search_wine':
State 'dict_keys(['documents'])':
("Value '{'documents': [Document(id='4a2f4d82-ea08-4f1d-9135-f28dbea4ef13', "
 "metadata={'source': '../data/restaurant_wine.txt', 'menu_number': 5, "
 "'menu_name': '푸이 퓌세 2019'}, page_content='5. 푸이 퓌세 2019\\n   • 가격: "
 '₩95,000\\n   • 주요 품종: 소비뇽 블랑\\n   • 설명: 프랑스 루아르 지역의 대표적인 화이트 와인입니다. 구스베리, '
 '레몬, 라임의 상큼한 과실향과 함께 미네랄, 허브 노트가 특징적입니다. 날카로운 산도와 깔끔한 피니시가 인상적이며, 신선한 굴이나 해산물 '
 "요리와 탁월한 페어링을 이룹니다.'), Document(id='a8848679-3674-4f5e-9855-fe63d5e01d70', "
 "metadata={'source': '../data/restaurant_wine.txt', 'menu_number': 2, "
 "'menu_name': '돔 페리뇽 2012'}, page_content='2. 돔 페리뇽 2012\\n   • 가격: "
 '₩380,000\\n   • 주요 품종: 샤르도네, 피노 누아\\n   • 설명: 프랑스 샴페인의 대명사로 알려진 프레스티지 큐베입니다. '
 '시트러스, 백도, 브리오쉬의 아로마가 조화롭게 어우러지며, 미네랄리티가 돋보입니다. 섬세하고 지속적인 버블과 크리미한 무스, 긴 여운이 '
 "특징입니다. 우아함과 복잡성이 완벽한 균형을 이룹니다.'), "
 "Document(id='24afee9f-1bd2-4315-bb8a-5f08c2a476c8', metadata={'source': "
 "'../data/restaurant_wine.txt', 'menu_number': 7, 'menu_name': '풀리니

### 3-2. 사람의 개입 (Human-in-the-Loop)

* Human-in-the-Loop (HITL)는 AI 시스템에 인간의 판단과 개입을 통합하는 접근 방식
* AI의 자동화된 처리와 인간의 전문성을 결합하여 더 정확하고 신뢰할 수 있는 결과를 도출하는 것을 목표
#### MemorySaver의 역할 
* MemorySaver 클래스는 LangGraph에서 **체크포인팅(Checkpointing)**을 구현하는 가장 기본적인 방법 중 하나로 사용됩니다. 
* 체크포인팅은 복잡한 워크플로우를 실행하는 데 있어 필수적인 기능입니다.
* 1. 상태 저장 및 복원 (Checkpointing)
    * MemorySaver의 주된 역할은 LangGraph의 **실행 상태(State)**를 저장하는 것입니다.
        * 저장 (Saving): 그래프의 실행이 특정 노드를 지날 때마다, 그 시점의 상태(AdaptiveRagState와 같은 딕셔너리)를 **메모리(RAM)**에 저장합니다.
        * 복원 (Restoring): 나중에 동일한 **스레드 ID (thread ID)**를 사용하여 그래프를 호출하면, MemorySaver는 이전에 저장된 마지막 상태를 불러와서 그 지점부터 실행을 재개할 수 있도록 합니다.
* 2. 메모리 기반 저장 (In-Memory)
    * MemorySaver는 데이터를 RAM에 저장합니다. 이는 매우 빠르다는 장점이 있지만, Python 세션이 종료되면 저장된 모든 체크포인트 데이터도 사라진다는 단점이 있습니다.         
    * 따라서 이는 주로 개발, 테스트, 데모 목적으로 사용되며, 프로덕션 환경이나 영구적인 상태 저장이 필요할 때는 SQLAlchemySaver와 같은 **영구적인 저장소(데이터베이스)**를 사용하는 체크포인트 구현체를 사용해야 합니다.
* 3. 대화 기록 유지 (Conversational Memory)
    * LangGraph가 실행의 연속성을 가지고 이전 상태를 기억하여 대화를 재개하거나 실패한 지점부터 작업을 다시 시작할 수 있도록 돕는 휘발성(Volatile) 메모리 기반의 상태 관리 도구입니다.    

`(1) 체크포인트 설정`

In [34]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

`(2) Breakpoint 추가`


In [35]:
# 컴파일 - 'generate' 노드 전에 중단점 추가
# ( builder는 이전에 정의된 StateGraph 객체이고, memory는 MemorySaver 객체이며, 'generate'는 답변 생성 노드의 이름입니다.)

# 컴파일 - 'generate' 노드 전에 중단점 추가
# LangGraph의 빌더(builder)를 최종 실행 가능한 그래프로 컴파일합니다.
adaptive_rag_hitl = builder.compile(
    checkpointer=memory,             # 1. 체크포인터 설정: 
                                     #    그래프의 모든 중간 상태를 'memory' 객체(MemorySaver)에 저장하도록 설정합니다.
                                     #    이를 통해 나중에 실행을 재개하거나 상태를 검토할 수 있습니다.
    interrupt_before=["generate"]    # 2. 인터럽트 설정 (Human-in-the-Loop 핵심):
                                     #    워크플로우가 'generate'라는 이름의 노드를 실행하기 직전에 멈추도록 지시합니다.
                                     #    이 중단점에서 사용자는 검색된 문서(documents)를 검토하거나 수정할 수 있습니다.
)


In [36]:

# 그래프 출력
#display(Image(adaptive_rag_hitl.get_graph().draw_mermaid_png()))
mermaid_code = adaptive_rag_hitl.get_graph().draw_mermaid()
print("Mermaid Code:")
print(mermaid_code)

Mermaid Code:
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	search_menu(search_menu)
	search_wine(search_wine)
	search_web(search_web)
	generate(generate<hr/><small><em>__interrupt = before</em></small>)
	llm_fallback(llm_fallback)
	__end__([<p>__end__</p>]):::last
	__start__ -.-> llm_fallback;
	__start__ -.-> search_menu;
	__start__ -.-> search_web;
	__start__ -.-> search_wine;
	search_menu --> generate;
	search_web --> generate;
	search_wine --> generate;
	generate --> __end__;
	llm_fallback --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



* https://mermaid.live/ 에서  mermain_code 로 직접 확인한다.

* [Graph이미지](https://mermaidchart.com/play?utm_source=mermaid_live_editor&utm_medium=share#pako:eNp9Ul2PmzAQ_CuW-0IkSIhJAjGUl-Yn3NMdVWRgHVCNQcaovUb577chH0fu2nvyjGZ3vTPaIy3aEiinByO6ijzt4kxndr_vrTD4OC9Jl95ZsujSnzPOuaxNb8-FPQhTVPsG9OBM8Gyi_a41OBP8oEHuvMNROYAGIyw4N5BUZpEmfSOUShNocJtaWzBm6Cz5TnKQrYFkgUKyuBSNY5Rq9hJZLopfzpTMLv5Al3d3I757U-Ji7e6aeHMvJdMZ8T_0ifsvZHT5lYrxxB9iJR7qtyzix-z-r-Ggz-KNjMrVdvwxrE9igYH0O5CkBCkGZYmsleLfJJO-lK7Cn7wK6kNl-XLOHhrGKxnLvbYTRW1fuf9QcI76Oi6X-UYW1MU7rEvKcZseXNqAacSZ02OmCcmoraCBjHKE13UymukT9nVCP7dtQ7k1A3aadjhUNzJ0Jdre1QKP_L0CLYL50Q7aUs624wTKj_QP5SE6CQOfrYPtZhmstmuXvlK-XEXzaMOiLVtHwSpk0cmlf8cv_XkYBixkbLkJmO8HYXR6A5WEKLE)

`(3) Breakpoint 실행 확인`
* LangGraph의 Human-in-the-Loop (HITL) 기능을 활용하여 워크플로우를 실행하고, 이전에 설정한 중단점에서 실행을 일시 정지시키는 과정을 보여줍니다. 
* MemorySaver와 thread_id를 사용하여 상태를 저장하고, stream을 통해 실시간으로 실행 흐름을 확인합니다.


In [37]:
# (Note: adaptive_rag_hitl은 'interrupt_before=["generate"]'로 컴파일된 LangGraph 객체입니다.)
from pprint import pprint

# 1. 스레드(Thread) 설정
# LangGraph 실행을 위한 설정(Config)을 정의합니다.
thread = {
    "configurable": {
        # 'thread_id'는 이 특정 실행의 고유 ID입니다. 
        # 이전에 설정한 MemorySaver(checkpointer)는 이 ID를 사용하여 
        # 그래프의 모든 중간 상태(체크포인트)를 저장하고 관리합니다. 
        # 이를 통해 나중에 멈춘 지점부터 정확하게 실행을 재개할 수 있습니다.
        "thread_id": "breakpoint_test"
    }
}

# 2. 입력 설정
# 그래프 실행에 사용할 사용자 질문을 딕셔너리 형태로 정의합니다.
inputs = {"question": "스테이크 메뉴의 가격은 얼마인가요?"}

# 3. 그래프 실행 및 스트리밍 (Execution and Streaming)
# adaptive_rag_hitl.stream()을 호출하여 그래프를 실행합니다.
# 'stream' 방식은 노드에서 노드로 이동할 때마다 이벤트를 반환하여 실시간 진행 상황을 보여줍니다.
# config=thread를 전달하여, 실행이 'breakpoint_test' 스레드에 속하도록 합니다.
for event in adaptive_rag_hitl.stream(inputs, config=thread):
    
    # 4. 이벤트 출력 루프
    # 각 이벤트는 {'노드_이름': {상태_업데이트}} 형태의 딕셔너리입니다.
    for k, v in event.items():
        
        # LangGraph는 실행이 완전히 끝났을 때 '__end__'라는 이벤트를 반환합니다.
        # 이 루프의 목적은 중단점 이전까지의 중간 과정을 확인하는 것이므로, 최종 이벤트는 건너뜁니다.
        if k != "__end__":
            # 현재 처리 중인 노드 이름(k)과 해당 노드가 반환한 상태 업데이트 값(v)을 출력합니다.
            # (예: 라우팅 결정, 검색 노드가 반환한 documents 리스트)
            pprint(f"{k}: {v}")  # 이벤트의 키와 값을 함께 출력
            
# 5. 실행의 최종 상태
# 이 코드는 실행 중 'generate' 노드 직전에 설정된 중단점에서 멈춥니다.
# 결과적으로 'search_menu' 노드까지의 실행 결과를 출력하고, 
# 'breakpoint_test' ID로 현재 상태가 체크포인트에 저장된 채로 종료됩니다.

("search_menu: {'documents': "
 "[Document(id='016ce496-a82b-43f0-bbac-49e2321459ef', metadata={'source': "
 "'../data/restaurant_menu.txt', 'menu_number': 1, 'menu_name': '시그니처 스테이크'}, "
 "page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,000\\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 "
 '감자, 그릴드 아스파라거스\\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 '
 '레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 '
 "풍부한 맛을 더합니다.'), Document(id='d6f94fb3-f07f-4524-b9c7-65f2dbc3e102', "
 "metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 8, "
 "'menu_name': '안심 스테이크 샐러드'}, page_content='8. 안심 스테이크 샐러드\\n   • 가격: "
 '₩26,000\\n   • 주요 식재료: 소고기 안심, 루꼴라, 체리 토마토, 발사믹 글레이즈\\n   • 설명: 부드러운 안심 '
 '스테이크를 얇게 슬라이스하여 신선한 루꼴라 위에 올린 메인 요리 샐러드입니다. 체리 토마토와 파마산 치즈 플레이크로 풍미를 더하고, '
 "발사믹 글레이즈로 마무리하여 고기의 풍미를 한층 끌어올렸습니다.'), "
 "Document(id='031767b9-b03e-4151-bf25-715429eeb5c7', metadata={'source': "
 "'../data/restaurant_menu.txt', 'menu_number': 2, 'menu_name': '트러플 리조또'}, "
 

`(4) Breakpoint 상태 관리`


In [47]:
# (Note: adaptive_rag_hitl은 컴파일된 LangGraph 객체이며, 
# thread는 이전에 실행을 시작했던 {"configurable": {"thread_id": "breakpoint_test"}} 딕셔너리입니다.)

# 1. 현재 그래프 상태 가져오기 (Get Current State)
# 'get_state' 메서드를 사용하여 특정 스레드 ID('breakpoint_test')에 저장된 
# 가장 최근의 체크포인트 상태(이 경우에는 중단점에서 멈춘 상태)를 불러옵니다.
current_state = adaptive_rag_hitl.get_state(thread)

# 2. 상태 출력 및 확인
print("---그래프 상태---")
# 불러온 current_state 객체는 LangGraph의 State Snapshot을 나타냅니다.
# 여기에는 질문, 검색된 문서, 실행 경로 등 모든 정보가 포함되어 있습니다.
print(current_state)
for doc in current_state.values['documents']:
    print(doc)
print("-"*50)

# 3. 특정 상태 값 확인 (Generation)
# current_state.values는 실제 상태 변수(딕셔너리)를 담고 있습니다.
# 'generation' 키의 값을 가져오려고 시도합니다.
# **중요:** 이 시점('generate' 노드 이전)에서는 'generation'이 아직 생성되지 않았으므로,
# 일반적으로 이 값은 None이거나 해당 키가 없을 수 있습니다. 
# 이 코드는 상태에 'generation'이 있는지 확인하려는 의도로 사용될 수 있습니다.
print(current_state.values.get("generation"))

---그래프 상태---
StateSnapshot(values={'question': '매콤한 해산물 요리가 있나요?', 'documents': [Document(id='7813772c-abfb-4d80-8a9c-e6297335359a', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 10, 'menu_name': '티라미수'}, page_content='10. 티라미수\n    • 가격: ₩9,000\n    • 주요 식재료: 마스카포네 치즈, 에스프레소, 카카오 파우더, 레이디핑거 비스킷\n    • 설명: 부드러운 마스카포네 치즈 크림과 에스프레소에 적신 레이디핑거 비스킷을 층층이 쌓아 만든 이탈리아 정통 디저트입니다. 고소한 카카오 파우더를 듬뿍 뿌려 풍미를 더했습니다. 커피의 쌉싸름함과 치즈의 부드러움이 조화롭게 어우러집니다.'), Document(id='df2475d9-3bac-4095-b3d4-7525f71c9cd7', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 9, 'menu_name': '치킨 콘피'}, page_content='9. 치킨 콘피\n   • 가격: ₩23,000\n   • 주요 식재료: 닭다리살, 허브, 마늘, 올리브 오일\n   • 설명: 닭다리살을 허브와 마늘을 넣은 올리브 오일에 저온에서 장시간 조리한 프랑스 요리입니다. 부드럽고 촉촉한 육질이 특징이며, 로즈메리 감자와 제철 채소를 곁들여 제공합니다. 레몬 제스트를 뿌려 상큼한 향을 더했습니다.'), Document(id='7a3eff30-b92e-4e93-a142-5093561aa681', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 4, 'menu_name': '버섯 크림 수프'}, page_content='4. 버섯 크림 수프\n   • 가격: ₩10,0

In [39]:
# 다음에 실행될 노드를 확인 
current_state.next

('generate',)

`(5) Breakpoint 이후 단계를 계속해서 실행`

In [51]:
# (Note: adaptive_rag_hitl은 이전에 'interrupt_before=["generate"]'로 컴파일된 LangGraph 객체입니다.)
# (thread는 이전에 실행을 멈췄던 스레드의 ID를 가진 {"configurable": {"thread_id": "breakpoint_test"}} 딕셔너리입니다.)

# 1. 중단된 실행 재개 및 스트리밍
# adaptive_rag_hitl.stream()을 다시 호출하여 실행을 재개합니다.
# 첫 번째 인수를 None으로 설정하는 것이 중요합니다:
# - None: 새로운 입력을 제공하지 않고, 이전에 'thread'에 저장된 상태를 기반으로 실행을 재개하라는 의미입니다.
# - config=thread: 이전에 중단되었던 'breakpoint_test' 스레드의 상태를 불러오도록 지정합니다.
for event in adaptive_rag_hitl.stream(None, config=thread):
    
    # 2. 이벤트 출력 루프
    # 재개된 실행의 이벤트(노드 실행 결과)를 실시간으로 확인합니다.
    for k, v in event.items():
        
        # LangGraph의 최종 종료 이벤트는 건너뜁니다.
        if k != "__end__":
            # 현재 실행 중인 노드 이름(k)과 해당 노드가 반환한 상태 업데이트 값(v)을 출력합니다.
            # 이 출력은 중단점 이후의 실행, 즉 'generate' 노드의 실행 결과를 보여줍니다.
            pprint(f"{k}: {v}")  # 이벤트의 키와 값을 함께 출력
            
# 3. 실행의 최종 상태
# 이 코드를 실행하면, 그래프는 이전에 멈췄던 지점('generate' 노드 직전)에서 시작하여,
# 'generate' 노드를 실행하고 최종 답변을 생성한 후, 'END' 노드로 이동하며 실행을 완료합니다.

In [50]:
# (Note: adaptive_rag_hitl은 컴파일된 LangGraph 객체이며, 
# thread는 이전에 실행을 멈췄던 스레드의 ID를 가진 딕셔너리입니다.)

# 1. 현재 그래프 상태 가져오기 (Get Current State)
# 'get_state' 메서드를 사용하여 특정 스레드 ID에 저장된 가장 최근의 체크포인트 상태를 불러옵니다.
# 이 상태는 이전에 'interrupt_before=["generate"]' 설정에 의해 멈춘 지점의 정보입니다.
current_state = adaptive_rag_hitl.get_state(thread)

# 2. 다음에 실행될 노드 확인
# 불러온 current_state 객체에는 현재 상태 외에도 다음에 실행될 노드의 이름이 저장되어 있습니다.
# 'next' 속성은 실행이 재개될 경우 가장 먼저 호출될 노드의 이름(들)을 리스트 형태로 반환합니다.
# 이 경우, 이전에 중단된 지점이 'generate' 노드 직전이었으므로, 출력은 ['generate']가 됩니다.
current_state.next
# print(current_state.next) # 예시 출력: ['generate']

()

In [ ]:
# (Note: adaptive_rag_hitl은 컴파일된 LangGraph 객체이며, 
# thread는 이전에 실행을 시작했던 스레드 ID를 가진 딕셔너리입니다.)

# 최종 답변 확인
# 1. 현재 그래프 상태 가져오기
# 'get_state' 메서드를 사용하여 이전에 중단되었거나 저장된 특정 스레드 ID의 
# 가장 최근 체크포인트 상태(State Snapshot)를 불러옵니다.
current_state = adaptive_rag_hitl.get_state(thread)
# key - question(질문), documents(tool이 호출한 결과), generation(최종결과)
print(current_state)

StateSnapshot(values={'question': '매콤한 해산물 요리가 있나요?', 'documents': [Document(id='7813772c-abfb-4d80-8a9c-e6297335359a', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 10, 'menu_name': '티라미수'}, page_content='10. 티라미수\n    • 가격: ₩9,000\n    • 주요 식재료: 마스카포네 치즈, 에스프레소, 카카오 파우더, 레이디핑거 비스킷\n    • 설명: 부드러운 마스카포네 치즈 크림과 에스프레소에 적신 레이디핑거 비스킷을 층층이 쌓아 만든 이탈리아 정통 디저트입니다. 고소한 카카오 파우더를 듬뿍 뿌려 풍미를 더했습니다. 커피의 쌉싸름함과 치즈의 부드러움이 조화롭게 어우러집니다.'), Document(id='df2475d9-3bac-4095-b3d4-7525f71c9cd7', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 9, 'menu_name': '치킨 콘피'}, page_content='9. 치킨 콘피\n   • 가격: ₩23,000\n   • 주요 식재료: 닭다리살, 허브, 마늘, 올리브 오일\n   • 설명: 닭다리살을 허브와 마늘을 넣은 올리브 오일에 저온에서 장시간 조리한 프랑스 요리입니다. 부드럽고 촉촉한 육질이 특징이며, 로즈메리 감자와 제철 채소를 곁들여 제공합니다. 레몬 제스트를 뿌려 상큼한 향을 더했습니다.'), Document(id='7a3eff30-b92e-4e93-a142-5093561aa681', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 4, 'menu_name': '버섯 크림 수프'}, page_content='4. 버섯 크림 수프\n   • 가격: ₩10,000\n   • 주요 식

In [ ]:
# 2. 'generation' 필드의 값 출력
# current_state.values는 상태 딕셔너리(question, documents 등)를 담고 있습니다.
# .get("generation")을 사용하여 최종 답변 필드의 값을 가져와 출력합니다.
# 주의: 이전에 'interrupt_before=["generate"]' 설정으로 멈췄다면, 
# 'generate' 노드가 실행되지 않았으므로, 이 값은 보통 None이 됩니다.
print(current_state.values.get("generation"))

제공된 문서에는 이 질문에 답변할 정보가 포함되어 있지 않습니다.  

문서에는 총 6개의 메뉴(티라미수, 치킨 콘피, 버섯 크림 수프, 랍스터 비스크, 가든 샐러드, 트러플 리조또)가 소개되어 있으나, **매콤한 해산물 요리**에 대한 설명은 없습니다.  

- **랍스터 비스크**는 해산물(랍스터)을 사용하지만, 매콤한 맛보다는 "진한 비스크 수프"와 "생크림으로 부드러운 질감"이 강조되었습니다.  
- 다른 메뉴들도 해산물 또는 매콤함과 직접적인 관련이 없습니다.


`(6) 상태 업데이트`

In [43]:
# 새로운 thread를 생성하고, 새로운 질문을 수행 
thread = {"configurable": {"thread_id": "breakpoint_update"}}
inputs = {"question": "매운 음식이 있나요?"}
for event in adaptive_rag_hitl.stream(inputs, config=thread):
    for k, v in event.items():
        if k != "__end__":
            print(f"{k}: {v}") 

search_menu: {'documents': [Document(id='7813772c-abfb-4d80-8a9c-e6297335359a', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 10, 'menu_name': '티라미수'}, page_content='10. 티라미수\n    • 가격: ₩9,000\n    • 주요 식재료: 마스카포네 치즈, 에스프레소, 카카오 파우더, 레이디핑거 비스킷\n    • 설명: 부드러운 마스카포네 치즈 크림과 에스프레소에 적신 레이디핑거 비스킷을 층층이 쌓아 만든 이탈리아 정통 디저트입니다. 고소한 카카오 파우더를 듬뿍 뿌려 풍미를 더했습니다. 커피의 쌉싸름함과 치즈의 부드러움이 조화롭게 어우러집니다.'), Document(id='df2475d9-3bac-4095-b3d4-7525f71c9cd7', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 9, 'menu_name': '치킨 콘피'}, page_content='9. 치킨 콘피\n   • 가격: ₩23,000\n   • 주요 식재료: 닭다리살, 허브, 마늘, 올리브 오일\n   • 설명: 닭다리살을 허브와 마늘을 넣은 올리브 오일에 저온에서 장시간 조리한 프랑스 요리입니다. 부드럽고 촉촉한 육질이 특징이며, 로즈메리 감자와 제철 채소를 곁들여 제공합니다. 레몬 제스트를 뿌려 상큼한 향을 더했습니다.'), Document(id='7a3eff30-b92e-4e93-a142-5093561aa681', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 4, 'menu_name': '버섯 크림 수프'}, page_content='4. 버섯 크림 수프\n   • 가격: ₩10,000\n   • 주요 식재료: 양송이버섯, 표고버섯, 생크림, 트러플 오일\n   • 설명: 양

In [44]:
# 다음에 실행될 노드를 확인 
current_state = adaptive_rag_hitl.get_state(thread)
current_state.next

('generate',)

In [45]:
# question, generation 필드 확인
current_state = adaptive_rag_hitl.get_state(thread)
print(current_state.values.get("question"))
print("-"*50)
print(current_state.values.get("generation"))

매운 음식이 있나요?
--------------------------------------------------
None


In [46]:
# 상태 업데이트 - 질문을 수정하여 업데이트
adaptive_rag_hitl.update_state(thread, {"question": "매콤한 해산물 요리가 있나요?"})

# 상태 확인
new_state = adaptive_rag_hitl.get_state(thread)

print(new_state.values.get("question"))
print("-"*50)
print(new_state.values.get("generation"))

매콤한 해산물 요리가 있나요?
--------------------------------------------------
None


In [ ]:
# 입력값을 None으로 지정하면 중단점부터 실행하고 최종 답변을 생성 
for event in adaptive_rag_hitl.stream(None, config=thread):
    for k, v in event.items():
        # '__end__' 이벤트는 미출력
        if k != "__end__":
            print(f"{k}: {v}")  # 이벤트의 키와 값을 함께 출력

generate: {'generation': "제공된 문서에는 이 질문에 답변할 정보가 포함되어 있지 않습니다.  \n\n문서에 언급된 해산물 요리는 '랍스터 비스크'뿐이며, 설명에는 매콤한 맛이나 파프리카를 사용한 풍미에 대한 구체적인 언급이 없습니다. 따라서 매콤한 해산물 요리 존재 여부를 확인할 수 없습니다.  \n\n(참고: 랍스터 비스크의 주요 식재료에 파프리카가 포함되나, 매운 맛과의 연관성은 명시되지 않음)"}


In [ ]:
# 최종 답변 확인
print(event["generate"]["generation"])

제공된 문서에는 이 질문에 답변할 정보가 포함되어 있지 않습니다.  

문서에 언급된 해산물 요리는 '랍스터 비스크'뿐이며, 설명에는 매콤한 맛이나 파프리카를 사용한 풍미에 대한 구체적인 언급이 없습니다. 따라서 매콤한 해산물 요리 존재 여부를 확인할 수 없습니다.  

(참고: 랍스터 비스크의 주요 식재료에 파프리카가 포함되나, 매운 맛과의 연관성은 명시되지 않음)
